In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%cd /content/drive/MyDrive/lmms-finetune
!python -m pip install -r requirements.txt
!python -m pip install --no-cache-dir --no-build-isolation flash-attn

/content/drive/.shortcut-targets-by-id/19S6OQ4_XJOLXuVCT5QTF-xnI63OjgVO0/lmms-finetune


In [ ]:
!sh ./example_scripts/example_image.sh

2024-08-02 07:06:31.769059: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-02 07:06:32.102111: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-02 07:06:32.193675: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-02 07:06:32.736241: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-02 07:06:35.577515: W tensorflow/compiler/tf2

# Inference

In [9]:
from transformers import AutoProcessor, AutoTokenizer, LlavaForConditionalGeneration

model_id = 'llava-hf/llava-interleave-qwen-0.5b-hf'
tokenizer = AutoTokenizer.from_pretrained(model_id)
processor = AutoProcessor.from_pretrained(model_id)
model = LlavaForConditionalGeneration.from_pretrained('/content/drive/MyDrive/lmms-finetune/checkpoints/llava-interleave-qwen-0.5b_lora-True_qlora-False').to(0)

In [36]:
import json
import os
import copy
from PIL import Image

IMAGE_ROOT = '/content/drive/MyDrive/lmms-finetune/data'
def image_preprocess(images):
  return [Image.open(os.path.join(IMAGE_ROOT, image)) for image in images]

def adjust_conversation(image_num, conversation):
  conversation = copy.deepcopy(conversation)
  for _ in range(image_num):
    conversation[0]['content'].append({"type": "image"})
  return conversation

def evaluate(file_path):
  with open(file_path, 'r') as f:
    data = json.load(f)
  results = []
  conversation_template = [
        {

            "role": "user",
            "content": [
                {"type": "text", "text": "Please write a report about the images."},
            ],
        },
    ]
  for i, patient in enumerate(data):
    print(i, '/', len(data))
    images = patient['image']
    patient_id = patient['id']
    conversation = adjust_conversation(len(images), conversation_template)
    images = image_preprocess(images)
    prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
    inputs = processor(prompt, images, return_tensors='pt').to(0)

    output = model.generate(**inputs, max_new_tokens=512, do_sample=True)
    result = processor.decode(output[0], skip_special_tokens=True)
    try:
      results.append({'id': patient_id, 'report': result[result.index('{'):]})
    except:
      print(patient_id, result)
  return results


In [25]:
results = evaluate('/content/drive/MyDrive/lmms-finetune/data/annotation_quiz_all_val.json')


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


0 / 296


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


1 / 296


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


294 / 296


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


295 / 296


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


In [29]:
with open('/content/drive/MyDrive/lmms-finetune/data/eval_results.json', 'w') as f:
  json.dump(results, f)

In [37]:
results = evaluate('/content/drive/MyDrive/lmms-finetune/data/annotation_quiz_all_test.json')

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


0 / 590


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


1 / 590


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


588 / 590


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


589 / 590


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


In [41]:
with open('/content/drive/MyDrive/lmms-finetune/data/test_results.json', 'w') as f:
  json.dump(results, f)